In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler

In [3]:
# generate training features from 2017 predicted probablities

In [9]:
high_prob_qrel = pd.read_csv("2017.high.txt", sep=" ", header=None)
col_names = ['rel', 'topicid', "Human_PM", "Animal_PM", "Not_PM", \
"Disease_Exact", "Disease_General","Disease_Specific", "Disease_Not",\
"Gene_Exact", "Gene_Missing", "Gene_Missing_Variant","Gene_Diff_Variant", \
"Demo_Match", "Demo_Notdiscussed", "Demo_Exclude" ,'none', 'docid']
high_prob_qrel.columns = col_names
del high_prob_qrel['none']

In [10]:
# we use retrieved results
retrieval_results_2017 = pd.read_csv("../../scripts/searching/2017.searching/2017.basic.query.result.txt",sep="\t")
retrieval_results_2017=retrieval_results_2017[['TOPIC_NO','ID', 'SCORE']]
retrieval_results_2017.columns = ['topicid', 'docid', 'score']

retrieval_results_2017 = retrieval_results_2017.merge(high_prob_qrel, on = ['topicid', 'docid'], how='left')
retrieval_results_2017.dropna(inplace=True)
retrieval_results_2017.rel = retrieval_results_2017.rel.astype(int)

scaler = MinMaxScaler()
retrieval_results_2017[['score']]=scaler.fit_transform(retrieval_results_2017[['score']])

In [12]:
with open ("../../../training.for.2018.txt", 'w') as f:
    
    for i in set(retrieval_results_2017.topicid):
        
        df_topic = retrieval_results_2017.loc[retrieval_results_2017.topicid == i]
        
        for index, rows in df_topic.iterrows():
            
            topicid = rows["topicid"]
            docid = rows["docid"]
            rel = rows["rel"]

            records = rows.to_dict()
            line = "{} qid:{} {} {} {} {} {} {} {} {} {} {} {} {} {} {} 15:{} #{}".format(rel, topicid, \
                  records["Human_PM"], records["Animal_PM"], records["Not_PM"], \
                  records["Disease_Exact"], records["Disease_General"],records["Disease_Specific"], records["Disease_Not"],\
                  records["Gene_Exact"], records["Gene_Missing"], records["Gene_Missing_Variant"],records["Gene_Diff_Variant"], \
                  records["Demo_Match"], records["Demo_Notdiscussed"], records["Demo_Exclude"], records['score'], \
                  docid)

            f.write(line + "\n")

In [13]:
# generate testing features from 2018 predicted probablities

In [17]:
high_prob_2018 = pd.read_csv("../../scripts/reranking/2018.high.features.csv")
retrieval_results_2018 = pd.read_csv("../../scripts/searching/2018.searching/2018.basic.query.result.txt",sep="\t")
high_prob_2018 = high_prob_2018.merge(retrieval_results_2018, left_on=['topicid', 'docid'], right_on=['TOPIC_NO', 'ID'], how='left')
high_prob_2018[['SCORE']] = scaler.fit_transform(high_prob_2018[['SCORE']])

In [19]:
# in 2018 retrieval result, there is no relevance score
# we add relevance score to evaluate in Ranklib
qrel_2018 = pd.read_csv("../../data/topics/2018qrel.txt",sep=" ",header=None)
qrel_2018.columns = ['topicid','q0','docid','rel']
qrel_2018 = qrel_2018[['topicid','docid','rel']]

# merge and fill na
high_prob_2018['topicid'] = high_prob_2018['topicid'].astype(int)
high_prob_2018['docid'] = high_prob_2018['docid'].astype(str)
qrel_2018['topicid'] = qrel_2018['topicid'].astype(int)
qrel_2018['docid'] = qrel_2018['docid'].astype(str)
high_prob_2018 = high_prob_2018.merge(qrel_2018, on = ["topicid","docid"],how="left")
high_prob_2018.fillna(0, inplace=True)
high_prob_2018['rel'] = high_prob_2018['rel'].astype(int)

In [22]:
with open ("../../../testing.for.2018.txt", 'w') as f:
    
    for i in set(high_prob_2018.topicid):
        
        df_topic = high_prob_2018.loc[high_prob_2018.topicid == i]
        
        for index, rows in df_topic.iterrows():
            
            topicid = rows["topicid"]
            docid = rows["docid"]
            rel = rows["rel"]

            records = rows.to_dict()
            line = "{} qid:{} 1:{} 2:{} 3:{} 4:{} 5:{} 6:{} 7:{} 8:{} 9:{} 10:{} 11:{} 12:{} 13:{} 14:{} 15:{} #{}".format(rel, topicid, \
                  records["Human_PM"], records["Animal_PM"], records["Not_PM"], \
                  records["Disease_Exact"], records["Disease_General"],records["Disease_Specific"], records["Disease_Not"],\
                  records["Gene_Exact"], records["Gene_Missing"], records["Gene_Missing_Variant"],records["Gene_Diff_Variant"], \
                  records["Demo_Match"], records["Demo_Notdiscussed"], records["Demo_Exclude"],records['SCORE'],\
                  docid)

            f.write(line + "\n")